In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:

data = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

# ilkleri görüntüleme
print(data.head())


In [ ]:
# Veri setinin bilgileri
print(data.info())



In [ ]:
#temel istatistik görüntüleme
print(data.describe())

In [ ]:
# Eksik değerleri kontrol edin
print(data.isnull().sum())

In [ ]:

# Hedef değişkeni ve özellikleri ayırmak için
#   'Class' sütunu hedef değişken 
X = data.drop('Class', axis=1)  # Özellikler
y = data['Class']  # Hedef değişken


In [ ]:
# Eğitim ve test setlerine ayırın
# Özellikleri normalize edin (isteğe bağlı fakat genellikle faydalıdır)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)


In [ ]:
# Modeli oluşturma ve eğitimi
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)

In [ ]:

# Sonuç değerlendirmesi (before smote)
print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
from imblearn.over_sampling import SMOTE

# SMOTE ile azınlık sınıfının veri artırımı ile geliştirme
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Yeniden model eğitimi
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_resampled, y_resampled)

# yeni tahmin
y_pred_resampled = model.predict(X_test)

# Sonuçları değerlendirmesi 2
print("Accuracy Score (SMOTE):", accuracy_score(y_test, y_pred_resampled))
print("Classification Report (SMOTE):\n", classification_report(y_test, y_pred_resampled))


In [ ]:
# Confusion Matrix (SMOTE sonrası)
cm_resampled = confusion_matrix(y_test, y_pred_resampled)
plt.figure(figsize=(10,7))
sns.heatmap(cm_resampled, annot=True, fmt='d', cmap='Reds', xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (After SMOTE)')
plt.show()